In [ ]:
import json
import pandas as pd
from transformers import pipeline
from sentence_transformers import CrossEncoder

# 1. JSON 파일 로드
with open("qa_output.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# JSON 데이터를 데이터프레임으로 변환
df_qa_test = pd.DataFrame(qa_data)

# 2. 이미 파인튜닝된 모델과 토크나이저 사용
# 모델과 토크나이저는 이미 로드된 상태로 가정 (model, tokenizer)
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)  # `device` 인수 제거

# 3. CrossEncoder 로드
cross_encoder_model_name = "BAAI/bge-reranker-v2-m3"  # CrossEncoder 모델 이름
cross_encoder = CrossEncoder(cross_encoder_model_name)

# 4. 예측값 생성 및 성능 평가 함수
def cross_encoder_evaluate_qa_dataset(df_qa_test):
    """
    QA 데이터셋에 대해 LLM으로 예측값을 생성하고 CrossEncoder로 유사도를 평가합니다.

    Args:
    df_qa_test (pd.DataFrame): 평가할 QA 데이터셋

    Returns:
    pd.DataFrame: 질문, 실제 답변, 예측 답변, 유사도 점수를 포함한 결과 데이터프레임
    """
    results = []

    for i in range(len(df_qa_test)):
        question = df_qa_test.iloc[i]['question']
        ground_truth = df_qa_test.iloc[i]['answer']

        # 파인튜닝된 모델을 사용해 예측값 생성
        prediction = llm_pipeline(question, max_new_tokens=100, num_return_sequences=1)[0]["generated_text"]

        # 유사도 점수 계산
        similarity = cross_encoder.predict([[ground_truth, prediction]])[0]

        # 결과 저장
        results.append({
            "index": i,
            "question": question,
            "ground_truth": ground_truth,
            "prediction": prediction,
            "similarity": f"{similarity:.3f}"
        })

    return pd.DataFrame(results)

# 5. 평가 수행
df_result_cross_encoder = cross_encoder_evaluate_qa_dataset(df_qa_test)




In [ ]:
import pandas as pd

# 결과 정리 함수 (인덱스와 유사도만 데이터프레임으로 반환)
def get_similarity_results(df_result):
    """
    평가 결과에서 인덱스와 유사도만 정리하여 데이터프레임으로 반환합니다.

    Args:
    df_result (pd.DataFrame): 평가 결과 데이터프레임

    Returns:
    pd.DataFrame: 인덱스와 유사도만 포함된 데이터프레임
    """
    # 필요한 열만 선택
    if "index" in df_result.columns and "similarity" in df_result.columns:
        selected_columns = ["index", "similarity"]
        df_similarity = df_result[selected_columns]
        return df_similarity
    else:
        raise KeyError("The required columns 'index' and 'similarity' are not found in the dataframe.")

# 평가 수행 (예시 데이터프레임 사용)
# df_qa_test: QA 테스트 데이터프레임
# cross_encoder_evaluate_qa_dataset: QA 평가 함수
# 예제 데이터를 사용하거나 아래 코드를 실행 전에 데이터 준비가 필요합니다.
try:
    # 평가 수행 결과
    df_result_cross_encoder = cross_encoder_evaluate_qa_dataset(df_qa_test)

    # 결과 데이터프레임 생성
    df_similarity_results = get_similarity_results(df_result_cross_encoder)

    # 결과 데이터프레임 출력
    df_similarity_results

except KeyError as e:
    print(f"Error: {e}")

except Exception as e:
    print(f"Unexpected error: {e}")


In [ ]:
df_similarity_results